<a href="https://colab.research.google.com/github/elichen/aoc2018/blob/main/Day_23_Experimental_Emergency_Teleportation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
data = """pos=<0,0,0>, r=4
pos=<1,0,0>, r=1
pos=<4,0,0>, r=3
pos=<0,2,0>, r=1
pos=<0,5,0>, r=3
pos=<0,0,3>, r=1
pos=<1,1,1>, r=1
pos=<1,1,2>, r=1
pos=<1,3,1>, r=1""".split('\n')

In [74]:
data = """pos=<10,12,12>, r=2
pos=<12,14,12>, r=2
pos=<16,12,12>, r=4
pos=<14,14,14>, r=6
pos=<50,50,50>, r=200
pos=<10,10,10>, r=5""".split('\n')

In [75]:
data = [x.rstrip() for x in open("input.txt").readlines()]

In [76]:
# Define the function to parse the string data into a list of positions and radius values

def parse_positions_and_radii(data):
    parsed_data = []
    for item in data:
        # Splitting each string at ', ' to separate the position and radius
        pos_str, r_str = item.split(", r=")

        # Extracting the coordinates from the position string
        pos = tuple(map(int, pos_str.lstrip('pos=<').rstrip('>').split(',')))

        # Converting the radius string to an integer
        r = int(r_str)

        # Adding the tuple of position and radius to the parsed data list
        parsed_data.append((pos, r))

    return parsed_data

parsed_data = parse_positions_and_radii(data)

In [77]:
def manhattan_distance_3d(point1, point2):
    return sum(abs(a - b) for a, b in zip(point1, point2))

strongest = max(parsed_data, key=lambda x:x[1])
total = 0
for r in parsed_data:
  if manhattan_distance_3d(r[0],strongest[0]) <= strongest[1]:
    total += 1
total

383

In [78]:
def count_items_in_optimized_range(items, x_range, y_range, z_range):
    """
    Optimized count of items in the list that are within a specified 3D range.
    The 'items' list contains tuples with positions and radii.
    The range is defined by x_range, y_range, and z_range.
    The radius is considered using Manhattan distance.
    """
    count = 0
    for (x, y, z), radius in items:
        # Calculate the Manhattan distance to the nearest point in the range
        distance_x = max(0, max(x_range[0] - x, x - x_range[1]))
        distance_y = max(0, max(y_range[0] - y, y - y_range[1]))
        distance_z = max(0, max(z_range[0] - z, z - z_range[1]))

        total_distance = distance_x + distance_y + distance_z

        # Check if the total Manhattan distance is within the radius
        if total_distance <= radius:
            count += 1

    return count

# example_range = ((-1000000000, 1000000000), (-1000000000, 1000000000), (-1000000000, 1000000000))
example_range = ((10,10),(10,10),(10,10))
count_items_in_optimized_range(parsed_data, *example_range)

56

In [79]:
from queue import PriorityQueue

def find_closest_point(items):
    def subdivide_range(x_range, y_range, z_range):
        """ Subdivide the 3D range into 8 quadrants. """
        x_mid = (x_range[0] + x_range[1]) // 2
        y_mid = (y_range[0] + y_range[1]) // 2
        z_mid = (z_range[0] + z_range[1]) // 2
        return [
            ((x_range[0], x_mid), (y_range[0], y_mid), (z_range[0], z_mid)),
            ((x_range[0], x_mid), (y_range[0], y_mid), (z_mid + 1, z_range[1])),
            ((x_range[0], x_mid), (y_mid + 1, y_range[1]), (z_range[0], z_mid)),
            ((x_range[0], x_mid), (y_mid + 1, y_range[1]), (z_mid + 1, z_range[1])),
            ((x_mid + 1, x_range[1]), (y_range[0], y_mid), (z_range[0], z_mid)),
            ((x_mid + 1, x_range[1]), (y_range[0], y_mid), (z_mid + 1, z_range[1])),
            ((x_mid + 1, x_range[1]), (y_mid + 1, y_range[1]), (z_range[0], z_mid)),
            ((x_mid + 1, x_range[1]), (y_mid + 1, y_range[1]), (z_mid + 1, z_range[1])),
        ]

    # Initialize the ranges
    x_min, x_max = min(x for (x, y, z), _ in items), max(x for (x, y, z), _ in items)
    y_min, y_max = min(y for (x, y, z), _ in items), max(y for (x, y, z), _ in items)
    z_min, z_max = min(z for (x, y, z), _ in items), max(z for (x, y, z), _ in items)

    # Initialize the priority queue with the initial range
    pq = PriorityQueue()
    initial_range = ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    initial_count = count_items_in_optimized_range(items, *initial_range)
    pq.put((-initial_count, initial_range))

    while not pq.empty():
        count, (x_range, y_range, z_range) = pq.get()

        # Check if the range is a single point
        if x_range[0] == x_range[1] and y_range[0] == y_range[1] and z_range[0] == z_range[1]:
            return (x_range[0], y_range[0], z_range[0])

        # Subdivide the range and add the sub-ranges to the queue
        for sub_range in subdivide_range(x_range, y_range, z_range):
            sub_count = count_items_in_optimized_range(items, *sub_range)
            pq.put((-sub_count, sub_range))

    # If no single point is found, return None (should not happen in this context)
    return None

# Test the function to find the closest point to the items
closest_point = find_closest_point(parsed_data)
closest_point

(35689633, 20484373, 44300020)

In [80]:
manhattan_distance_3d((0,0,0), closest_point)

100474026